# Dog breed prediction in Google Cloud Machine Learning

The following cells create a Keras model merging Inception V3 and our top model created in project 2.
The input of Inception V3 is modified in order to accept the images encoded as base64 1-D vector which is more apropiate in internet transfers. 
This modified model is saved in a folder in the 'pb' tensorflow format that Google expect.
Next step link this folder with a new registered Google ML model. Now you can request predictions to this model.
The output is still a 1D vector of 133 values that represents our classes. The table of dog_names will give us the right breed names.

## Warning:
- This notebook only works in Python 2
- A Google Cloud account and the instalation of the corresponding Python SDK are required.

## Notes:
Thanks to Hayato Yoshikawa in https://medium.com/google-cloud/serverless-transfer-learning-with-cloud-ml-engine-and-keras-335435f31e15 for detailed explanations in how to proceed. Also in https://github.com/hayatoy/cloudml-magic for the code to automate the deployment in Google ML.
A modified version is used here.

In [1]:
%load_ext cloudmlmagic2

In [2]:
%ml_init -projectId mezquita-dog-breed -bucket mezquita-dog-breed/mezquita-dog-breed -requires keras h5py Pillow -scaleTier BASIC -region europe-west1 -runtimeVersion 1.2

## Load the top model pretrained

In [3]:
%%ml_code
from keras.utils.data_utils import get_file
from keras.models import load_model
model_path = get_file(
            'my_model.hdf5',
            'https://github.com/gregoriomezquita/dog-breed/raw/master/cloud-ml/my_model.hdf5')
my_model = load_model(model_path)

Using TensorFlow backend.


## Load the inception V3 model

In [4]:
%%ml_code

from keras.applications.inception_v3 import InceptionV3 as keras_model

selected_model= keras_model(weights='imagenet', include_top=False)

## Combine both models

In [5]:
%%ml_code

from keras.models import Model
# Concatenate both models
final_model = Model(inputs= [selected_model.input], outputs= [my_model(selected_model.output)])

## Deploy the combined model in Google Storage ready to predict

In [6]:
%%ml_run cloud

import tensorflow as tf
from keras import backend as K

saved_model_path= 'gs://mezquita-dog-breed/mezquita-dog-breed/savedmodel'
#saved_model_path= 'savedmodel'

K.set_learning_phase(0)  # test
sess = K.get_session()

from tensorflow.python.framework import graph_util

# Make GraphDef of Transfer Model
g_trans = sess.graph
g_trans_def = graph_util.convert_variables_to_constants(sess,
                                                        g_trans.as_graph_def(),
                                                        [final_model.output.name.replace(':0','')])

# Image Converter Model
with tf.Graph().as_default() as g_input:
    input_b64 = tf.placeholder(shape=(1,), dtype=tf.string, name='input')
    input_bytes = tf.decode_base64(input_b64[0])
    image = tf.image.decode_image(input_bytes)
    image_f = tf.image.convert_image_dtype(image, dtype=tf.float32)
    input_image = tf.expand_dims(image_f, 0)
    output = tf.identity(input_image, name='input_image')

g_input_def = g_input.as_graph_def()



with tf.Graph().as_default() as g_combined:
    x = tf.placeholder(tf.string, name="input_b64")

    im, = tf.import_graph_def(g_input_def,
                              input_map={'input:0': x},
                              return_elements=["input_image:0"])

    pred, = tf.import_graph_def(g_trans_def,
                                input_map={final_model.input.name: im,
                                          'batch_normalization_1/keras_learning_phase:0': False},
                                return_elements=[final_model.output.name])

    with tf.Session() as sess2:
        inputs = {"inputs": tf.saved_model.utils.build_tensor_info(x)}
        outputs = {"outputs": tf.saved_model.utils.build_tensor_info(pred)}
        signature = tf.saved_model.signature_def_utils.build_signature_def(
            inputs=inputs,
            outputs=outputs,
            method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
        )

        # save as SavedModel
        b = tf.saved_model.builder.SavedModelBuilder(saved_model_path)
        b.add_meta_graph_and_variables(sess2,
                                       [tf.saved_model.tag_constants.SERVING],
                                       signature_def_map={'serving_default': signature})
        b.save()

{u'trainingOutput': {}, u'trainingInput': {u'runtimeVersion': u'1.2', u'packageUris': [u'gs://mezquita-dog-breed/mezquita-dog-breed/mlmagic__1516809824.tar.gz'], u'region': u'europe-west1', u'pythonModule': u'trainer.task'}, u'jobId': u'mlmagic__1516809824', u'state': u'QUEUED', u'etag': u'1TCtVvYpsJs=', u'createTime': u'2018-01-24T16:04:05Z'}


## Important !!!!
Wait until the task created finishes.

## Create model in Google Machine Learning
The new model is associated with the files deplyed bellow

In [11]:
#!gcloud ml-engine models create DogBreed --project mezquita-dog-breed --regions europe-west1
!gcloud ml-engine versions create v1 --model DogBreed --origin gs://mezquita-dog-breed/mezquita-dog-breed/savedmodel --project mezquita-dog-breed --runtime-version 1.2 

Creating version (this might take a few minutes)......done.                    


## Time to use the service.

In [1]:

from oauth2client.client import GoogleCredentials
from googleapiclient import discovery
from googleapiclient import errors

PROJECTID = 'mezquita-dog-breed'
projectID = 'projects/{}'.format(PROJECTID)
modelName = 'DogBreed'
modelID = '{}/models/{}'.format(projectID, modelName)

credentials = GoogleCredentials.get_application_default()
ml = discovery.build('ml', 'v1', credentials=credentials)
vision = discovery.build('vision', 'v1', credentials=credentials)

In [14]:
import base64
import json
import os.path
import numpy as np
from io import BytesIO
from keras.preprocessing import image

def load_dog_names(file_name):
    with open(file_name, 'r') as f:
        return json.load(f)
    
def predict_raw(data):
    b64_x= base64.urlsafe_b64encode(data)
    input_instance=  dict(inputs= b64_x)
    
    request = ml.projects().predict( name= modelID, body= dict(instances= [input_instance]) )
    try:
        response = request.execute()
    except errors.HttpError as err:
        # Something went wrong with the HTTP transaction.
        # To use logging, you need to 'import logging'.
        print('There was an HTTP error during the request:')
        print(err._get_reason())

    if response.get('error') != None: print(response)
    else:
        predictions= response.get('predictions')[0].get('outputs')
        max= np.argmax(predictions)
        accuracy= predictions[max]* 100
        return dog_names[max], accuracy
    return '', 0.0

def predict(img_path):
    extension = os.path.splitext(img_path)[1][1:].upper()
    if extension== 'JPG': extension= 'JPEG'
    img = image.load_img(img_path, target_size=(224, 224))
    buf= BytesIO()
    img.save(buf, format= extension)
    return predict_raw(buf.getvalue())

dog_names= load_dog_names('dog_names.json')

## Test with images

In [17]:
## Test prediction service with the images folder
#
from glob import glob

files= sorted(glob("../test/*"))
for img_path in files:
    breed, acc= predict(img_path)
    print('%35s : %s (%.2f)' % (img_path[8:], breed, acc))


                       angelina.jpg : Dalmatian (30.81)
                         begle.jpeg : Beagle (100.00)
                           cat.jpeg : Nova scotia duck tolling retriever (95.55)
                     chihuahua.jpeg : Chihuahua (100.00)
                          male1.jpg : Dalmatian (24.40)
                          trump.jpg : Komondor (63.90)


## Using Google App Engine to make predictions

Random dog pictures used.

{"breed": "Xoloitzcuintli", "result": "Dog", "accuracy": "99.82 %"}


In [9]:
## Test prediction service with random dog pictures
#
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt                        
%matplotlib inline
import json

with open('dog_names.json', 'r') as f: dog_names= json.load(f)
    
r = requests.get('https://dog.ceo/api/breeds/image/random')
if r.status_code == requests.codes.ok:  # image returned OK
    img_url= json.loads(r.content).get('message')
    r.close() 
    r = requests.get(img_url)
    if r.status_code == requests.codes.ok:
        image = Image.open(BytesIO(r.content))
        
        r= requests.post(url= 'http://mezquita-dog-breed.appspot.com/predict',
                         files= {'image': r.content})
        print(r.content)
    plt.imshow(image)
    plt.show()



## Easy


In [19]:
# OS curl command
#
!curl -F 'image=@../test/chihuahua.jpeg' https://mezquita-dog-breed.appspot.com/breed

{"breed": "Chihuahua", "accuracy": 99.999547004699707}

In [20]:
# Python way
#
def curl_F(endpoint, img_path):
    with open(img_path, 'rb') as face_file: content = face_file.read()
    r= requests.post(url= 'http://mezquita-dog-breed.appspot.com/'+ endpoint,
                         files= {'image': content})
    return r.content

curl_F('breed', '../test/chihuahua.jpeg')

'{"breed": "Chihuahua", "accuracy": 99.999547004699707}'